# Ames Housing Project Suggestions

Data science is not a linear process. In this project, in particular, you will likely find that EDA, data cleaning, and exploratory visualizations will constantly feed back into each other. Here's an example:

1. During basic EDA, you identify many missing values in a column/feature.
2. You consult the data dictionary and use domain knowledge to decide _what_ is meant by this missing feature.
3. You impute a reasonable value for the missing value.
4. You plot the distribution of your feature.
5. You realize what you imputed has negatively impacted your data quality.
6. You cycle back, re-load your clean data, re-think your approach, and find a better solution.

Then you move on to your next feature. _There are dozens of features in this dataset._

Figuring out programmatically concise and repeatable ways to clean and explore your data will save you a lot of time.

The outline below does not necessarily cover every single thing that you will want to do in your project. You may choose to do some things in a slightly different order. Many students choose to work in a single notebook for this project. Others choose to separate sections out into separate notebooks. Check with your local instructor for their preference and further suggestions.

## EDA
- **Read the data dictionary.**
- Determine _what_ missing values mean.
- Figure out what each categorical value represents.
- Identify outliers.
- Consider whether discrete values are better represented as categorical or continuous. (Are relationships to the target linear?)

## Data Cleaning
- Decide how to impute null values.
- Decide how to handle outliers.
- Do you want to combine any features?
- Do you want to have interaction terms?
- Do you want to manually drop collinear features?

## Exploratory Visualizations
- Look at distributions.
- Look at correlations.
- Look at relationships to target (scatter plots for continuous, box plots for categorical).

## Pre-processing
- One-hot encode categorical variables.
- Train/test split your data.
- Scale your data.
- Consider using automated feature selection.

## Modeling
- **Establish your baseline score.**
- Fit linear regression. Look at your coefficients. Are any of them wildly overblown?
- Fit lasso/ridge/elastic net with default parameters.
- Go back and remove features that might be causing issues in your models.
- Tune hyperparameters.
- **Identify a production model.** (This does not have to be your best performing Kaggle model, but rather the model that best answers your problem statement.)
- Refine and interpret your production model.

## Inferential Visualizations
- Look at feature loadings.
- Look at how accurate your predictions are.
- Is there a pattern to your errors? Consider reworking your model to address this.

## Business Recommendations
- Which features appear to add the most value to a home?
- Which features hurt the value of a home the most?
- What are things that homeowners could improve in their homes to increase the value?
- What neighborhoods seem like they might be a good investment?
- Do you feel that this model will generalize to other cities? How could you revise your model to make it more universal OR what date would you need from another city to make a comparable model?

# Example Directory Structure
Here's how you might structure a project with multiple notebooks.

```
project-2
|__ code
|   |__ 01_EDA_and_Cleaning.ipynb   
|   |__ 02_Preprocessing_and_Feature_Engineering.ipynb   
|   |__ 03_Model_Benchmarks.ipynb
|   |__ 04_Model_Tuning.ipynb  
|   |__ 05_Production_Model_and_Insights.ipynb
|   |__ 06_Kaggle_Submissions.ipynb   
|__ data
|   |__ train.csv
|   |__ test.csv
|   |__ submission_lasso.csv
|   |__ submission_ridge.csv
|__ images
|   |__ coefficients.png
|   |__ neighborhoods.png
|   |__ predictions.png
|__ presentation.pdf
|__ README.md
```

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [216]:
#Get datasets: NB, test is just read but will not be written over until modelling is done
#test = '../datasets/test.csv'
train = '../datasets/train.csv'

In [217]:
df = pd.read_csv(train)

In [4]:
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [237]:
# Checking for nulls
df.isnull().sum().sort_values().tail(15)

##no. nulls similar columns might be related.

##From data dictionary, there are columns with NA columns to signify the absence of the additional ammenities
# Pandas dataframe automatically considers all NA columns as NaN, whereas the columns are actually a category by itself
#Need to check and replace all columns that are not in actual fact <blank>/missing

Foundation           0
Mas Vnr Type         0
Exterior 2nd         0
Exterior 1st         0
Roof Matl            0
Roof Style           0
Year Remod/Add       0
Mas Vnr Area         0
SalePrice            0
Lot Frontage       330
Fireplace Qu      1000
Fence             1651
Alley             1911
Misc Feature      1986
Pool QC           2042
dtype: int64

In [218]:
# If BsmtFin SF 1, SF 2 and total bsmt SF are NaN, total square feet should be 0.
df[df['BsmtFin SF 1'].isnull()][['BsmtFin SF 1','Total Bsmt SF', 'BsmtFin SF 2', 'Bsmt Unf SF']]

,BsmtFin SF 1,Total Bsmt SF,BsmtFin SF 2,Bsmt Unf SF
1327,NaN,NaN,NaN,NaN


In [219]:
df=df.fillna({'BsmtFin SF 1':0, 'Total Bsmt SF': 0, 'BsmtFin SF 2':0,'Bsmt Unf SF':0})

In [220]:
#If Garage area and Garage Cars are NaN, absence of garage area should set to 'None'.
# Garage car in number of cars should be set to integer '0' if NaN.
df[df['Garage Area'].isnull()][['Garage Cars']]

,Garage Cars
1712,NaN


In [221]:
# If 
df=df.fillna({'Garage Area':'None','Garage Cars':0})

In [222]:
# Bsmt half or full bathroom, if NaN --> Set to 0.0
df=df.fillna({'Bsmt Half Bath':0.0, 'Bsmt Full Bath': 0.0})

In [223]:
#Masonry veneer type if NaN, set to None
#Masonry veneer area if NaN, set to '0'
df=df.fillna({'Mas Vnr Type':'None', 'Mas Vnr Area': 0.0})

In [224]:
# BsmtFin Type 1 and Type 2, Bsmt Qual, Cond and Exposure if NaN, set to "None"
df=df.fillna({'BsmtFin Type 1':'None',
              'BsmtFin Type 2':'None',
              'Bsmt Qual':'None',
              'Bsmt Cond':'None',
              'Bsmt Exposure':'None'
             })

In [225]:
#isnull for Garage type != Garage qual/cond/fin ----> 113 != 114;
#There is a row with garage type = Detchd but no quality, condition, finish and yr Belt
df[df['Garage Qual'].isnull()]['Garage Type'].sort_values(ascending = True).head(1)

1712    Detchd
Name: Garage Type, dtype: object

In [226]:
#Show mode of Garage Qual, Cond, Finish and Yr Blt when Garage type == Detchd
garage = df[df['Garage Type'] == 'Detchd'].mode()
garage[['Garage Qual','Garage Cond','Garage Finish', 'Garage Yr Blt']].head(3)

,Garage Qual,Garage Cond,Garage Finish,Garage Yr Blt
0,TA,TA,Unf,1920.0
1,NaN,NaN,NaN,1930.0
2,NaN,NaN,NaN,NaN


In [227]:
#Replace garage quality, condition, finish and year built of index 1712 with mode of each column
df.loc[1712,'Garage Qual'] = 'TA'
df.loc[1712,'Garage Cond'] = 'TA'
df.loc[1712,'Garage Finish'] = 'Unf'
df.loc[1712,'Garage Yr Blt'] = '1920.0'

In [228]:
# Garage Type, Qual, Cond, Finish set to 'None' if NaN
#Garage Yr Blt set to 0.0 if NaN
df=df.fillna({'Garage Type':'None',
              'Garage Qual':'None',
              'Garage Cond':'None',
              'Garage Finish':'None',
              'Garage Yr Blt':0.0
             })

In [229]:
#string detected in garage yr blt; dtype = obj
df['Garage Yr Blt'].dtypes

#TypeError when converting Garage Yr Blt to float

dtype('O')

TypeError: '<' not supported between instances of 'float' and 'str'

In [233]:
counter = 0
for i in df['Garage Yr Blt']:
    if not isinstance(i,str):
        counter += 1
    else:
        print('The incorrect row prints', i ,'as string')
        print('The index number is:',counter)

The incorrect row prints 1920.0 as string
The index number is: 1712


In [234]:
#Correcting year as string in Garage Yr Blt
df.loc[1712,'Garage Yr Blt'] = 1920.0

In [235]:
#Unusual data year 2207; to correct to 2007
df['Garage Yr Blt'].sort_values(ascending = False).head()

1699    2207
1241    2010
144     2010
1732    2009
1700    2009
Name: Garage Yr Blt, dtype: int64

In [236]:
#Correcting year 2207 as 2007
df.loc[1699,'Garage Yr Blt'] = 2007.0

## EDA
- **Read the data dictionary.**
- Determine _what_ missing values mean.
- Figure out what each categorical value represents.
- Identify outliers.
- Consider whether discrete values are better represented as categorical or continuous. (Are relationships to the target linear?)

In [97]:
#check for outliers and extreme values
df.skew().sort_values()

Year Built         -0.607913
Year Remod/Add     -0.451205
Garage Yr Blt      -0.267173
Garage Cars        -0.227820
Id                 -0.011139
PID                 0.064336
Full Bath           0.106913
Overall Qual        0.148461
Yr Sold             0.154255
Garage Area         0.199241
Mo Sold             0.212035
Bedroom AbvGr       0.370480
Bsmt Full Bath      0.630856
Overall Cond        0.638166
Fireplaces          0.726038
Half Bath           0.742920
TotRms AbvGrd       0.843940
2nd Flr SF          0.874577
Bsmt Unf SF         0.908480
Gr Liv Area         1.281492
MS SubClass         1.381004
Total Bsmt SF       1.388913
SalePrice           1.557551
BsmtFin SF 1        1.603090
1st Flr SF          1.635146
Lot Frontage        1.811116
Wood Deck SF        2.017081
Open Porch SF       2.298022
Mas Vnr Area        2.594917
Enclosed Porch      2.864913
Screen Porch        3.859110
Bsmt Half Bath      3.946994
BsmtFin SF 2        4.239955
Kitchen AbvGr       4.348274
Lot Area      

In [ ]:
#outlier of miscellaneous feature
print(df['Misc Val'].describe())
df['Misc Val'].sort_values(ascending = False).head(10)

In [ ]:
#Drop outliers > mean + 6.s.d (3491.91) 
df.drop([1885,304,765,1225,1786,380,700],inplace=True)

## Data Cleaning
- Decide how to impute null values.
- Decide how to handle outliers.
- Do you want to combine any features?
- Do you want to have interaction terms?
- Do you want to manually drop collinear features?


## Exploratory Visualizations
- Look at distributions.
- Look at correlations.
- Look at relationships to target (scatter plots for continuous, box plots for categorical).


## Pre-processing
- One-hot encode categorical variables.
- Train/test split your data.
- Scale your data.
- Consider using automated feature selection.

## Modeling
- **Establish your baseline score.**
- Fit linear regression. Look at your coefficients. Are any of them wildly overblown?
- Fit lasso/ridge/elastic net with default parameters.
- Go back and remove features that might be causing issues in your models.
- Tune hyperparameters.
- **Identify a production model.** (This does not have to be your best performing Kaggle model, but rather the model that best answers your problem statement.)
- Refine and interpret your production model.


## Inferential Visualizations
- Look at feature loadings.
- Look at how accurate your predictions are.
- Is there a pattern to your errors? Consider reworking your model to address this.

## Business Recommendations
- Which features appear to add the most value to a home?
- Which features hurt the value of a home the most?
- What are things that homeowners could improve in their homes to increase the value?
- What neighborhoods seem like they might be a good investment?
- Do you feel that this model will generalize to other cities? How could you revise your model to make it more universal OR what date would you need from another city to make a comparable model?